#### Importación de paquetes

In [13]:
import cv2
import FaceDetectors

#### Aplicación de filtro divertido con detector facial basado en Convolutional Neural Networks (CNNs)

In [14]:
# Inicializar el detector facial
FDet = FaceDetectors.FaceDetector()

# Cargar las imágenes de los filtros con transparencia (bigote, gafas, pelo)
bigote = cv2.imread('Images/bigote.png', cv2.IMREAD_UNCHANGED)
gafas = cv2.imread('Images/gafas.png', cv2.IMREAD_UNCHANGED)
pelo = cv2.imread('Images/pelo.png', cv2.IMREAD_UNCHANGED)

# Redimensionar las imágenes de los filtros para que se ajusten mejor al rostro
gafas = cv2.resize(gafas, (gafas.shape[1] // 4, gafas.shape[0] // 4))
pelo = cv2.resize(pelo, (pelo.shape[1] // 2, pelo.shape[0] // 2))

# Conectar a la cámara web
cap = cv2.VideoCapture(0)

# Verificar si hay otras cámaras disponibles
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Error de cámara')
            exit(0)
        else:
            print('Cámara 0')
    else:
        print('Cámara 1')
else:
    print('Cámara 0')

# Establecer la resolución de la cámara
cap.set(3, 640)
cap.set(4, 480)

def superponer_filtro(frame, filtro, shape_index, x_offset, y_offset):
    x_filtro = shape[shape_index][0] - int(filtro.shape[1] / 2)
    y_filtro = shape[shape_index][1] - int(filtro.shape[0] / 2) + y_offset

    if x_filtro >= 0 and y_filtro >= 0 and (x_filtro + filtro.shape[1] <= frame.shape[1]) and (y_filtro + filtro.shape[0] <= frame.shape[0]):
        filtro_resized = cv2.resize(filtro, (int(filtro.shape[1]), int(filtro.shape[0])))
        alpha = filtro_resized[:, :, 3] / 255.0

        for c in range(0, 3):
            frame[y_filtro:y_filtro + filtro_resized.shape[0], x_filtro:x_filtro + filtro_resized.shape[1], c] = (
                frame[y_filtro:y_filtro + filtro_resized.shape[0], x_filtro:x_filtro + filtro_resized.shape[1], c] * (1 - alpha) +
                filtro_resized[:, :, c] * alpha
            )

while True:
    # Capturar un fotograma de la cámara
    ret, frame = cap.read()

    # Detección de rostro y ojos en el fotograma
    values = FDet.DetectLargestFaceEyesDNN(frame)
    if values is not None:
        face, eyes, shape = values

        # Dibujar un marco alrededor del rostro
        [x, y, w, h] = face
        if x > -1:
            [lex, ley, rex, rey] = eyes
            if lex > -1:
                superponer_filtro(frame, bigote, 51, 0, 0)
                superponer_filtro(frame, gafas, 27, 0, 10)
                superponer_filtro(frame, pelo, 27, 0, -80)

    cv2.imshow('Cam', frame)
    
    # Esc para finalizar
    tec = cv2.waitKey(40)
    if tec == 27:  # Esc
        break

# Cerrar la ventana y liberar la cámara
cap.release()
cv2.destroyAllWindows()


Cámara 0
